In [36]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import string
import pickle

In [38]:
def remove_punctuation(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    return text


In [39]:
with open('../static/model/corpora/stopwords/english', 'r') as f:
    stopwords = f.read().splitlines()

In [40]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()

In [ ]:
vocab = pd.read_csv('../static/model/vocab.txt', header=None)
tokens = vocab[0].tolist()

In [ ]:
with open('../static/model/model.pickle', 'rb') as f:
    model = pickle.load(f)

In [41]:
def preprocess(text):
    data = pd.DataFrame([text], columns=['tweet'])
    data['tweet'] = data['tweet'].apply(lambda x: " ".join(x.lower() for x in x.split()))
    data['tweet'] = data['tweet'].apply(lambda x: " ".join(re.sub(r'^https?:\/\/.*[\r\n]*', '', x, flags=re.MULTILINE) for x in x.split()))
    data['tweet'] = data['tweet'].apply(remove_punctuation)
    data['tweet'] = data['tweet'].str.replace('\d+', '', regex=True)
    data['tweet'] = data['tweet'].apply(lambda x: " ".join(x for x in x.split() if x not in stopwords))
    data['tweet'] = data['tweet'].apply(lambda x: " ".join(ps.stem(x) for x in x.split()))
    return data['tweet']

In [42]:
preprocess_txt = preprocess(txt)

In [47]:
def vectorizer(ds, vocab):
    vectorized_lst = []
    for sentence in ds:
        sentence_list = np.zeros(len(vocab))
        
        for i in range(len(vocab)):
            if vocab[i] in sentence.split():
                sentence_list[i] = 1
        
        vectorized_lst.append(sentence_list)
        
    vector_new = np.asarray(vectorized_lst, dtype=np.float32)
    
    return vector_new

In [52]:
def get_prediction(vectorized_txt):
    prediction = model.predict(vectorized_txt)
    if prediction == 1:
        return 'Positive'
    else:
        return 'Negative'

In [61]:
txt = 'i am not happy today'
preprocess_txt = preprocess(txt)
vectorized_txt = vectorizer(preprocess_txt, tokens)
prediction = get_prediction(vectorized_txt)
prediction

'Negative'